In [1]:
import numpy
import pandas


In [2]:
index_date = pandas.date_range("2016-01-01", "2018-12-31")
index_date

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', length=1096, freq='D')

In [3]:
index_date.astype(numpy.int32)

Int64Index([1451606400000000000, 1451692800000000000, 1451779200000000000,
            1451865600000000000, 1451952000000000000, 1452038400000000000,
            1452124800000000000, 1452211200000000000, 1452297600000000000,
            1452384000000000000,
            ...
            1545436800000000000, 1545523200000000000, 1545609600000000000,
            1545696000000000000, 1545782400000000000, 1545868800000000000,
            1545955200000000000, 1546041600000000000, 1546128000000000000,
            1546214400000000000],
           dtype='int64', length=1096)

In [4]:
index_date.month

Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
            ...
            12, 12, 12, 12, 12, 12, 12, 12, 12, 12],
           dtype='int64', length=1096)

In [5]:
index_date.weekday

Int64Index([4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
            ...
            5, 6, 0, 1, 2, 3, 4, 5, 6, 0],
           dtype='int64', length=1096)

In [6]:
def get_datedata(start="2016-01-01", end="2018-12-31"):
    _df = pandas.DataFrame([])
    index_date = pandas.date_range(start, end)
    _df["month"] = index_date.month
    _df["weekth"] = index_date.week
    _df["weekday"] = index_date.weekday
    _df.month = _df.month.astype("category")
    _df.weekth = _df.month.astype("category")
    _df.weekday = _df.month.astype("category")
    df = pandas.get_dummies(_df, ["month", "weekth", "weekday"])
    df.index = index_date
    df["date_index"] = index_date.astype(numpy.int64)
    return df

df = get_datedata()
df.head(3)



,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,weekday_4,weekday_5,weekday_6,weekday_7,weekday_8,weekday_9,weekday_10,weekday_11,weekday_12,date_index
2016-01-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1451606400000000000
2016-01-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1451692800000000000
2016-01-03,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1451779200000000000


In [16]:
def get_datedata(start="2016-01-01", end="2018-12-31"):
    df = pandas.DataFrame([])
    index_date = pandas.date_range(start, end)
    df["date_index"] = index_date.astype(numpy.int64)
    return df

df = get_datedata()
df.head()


,date_index
0,1451606400000000000
1,1451692800000000000
2,1451779200000000000
3,1451865600000000000
4,1451952000000000000


In [7]:
import torch
def generate_window(df, wsz=4) -> torch.Tensor:
    n = len(df)
    for idx in range(0, n-wsz+1):
        bch = df.values[idx:idx+wsz]
        yield torch.Tensor(bch)

In [17]:
# ti_debug = []
# for bch in generate_window(df, wsz=4):
#     t0 = bch[-1, -1]  # last of seq should be used as current
#     ti_debug.append(t0.item())
# ti_debug = numpy.array(ti_debug)


In [18]:
df.head(3)

,date_index
0,1451606400000000000
1,1451692800000000000
2,1451779200000000000


In [22]:
ti_debug = []
for bch in generate_window(df, wsz=4):
    t = bch[-1, -1]  # last of seq should be used as current
    ti_debug.append(t.item())
ti_debug = numpy.array(ti_debug)
ti_debug




array([1.45186566e+18, 1.45195197e+18, 1.45203842e+18, ...,
       1.54604158e+18, 1.54612803e+18, 1.54621434e+18])

In [26]:
pandas.to_datetime(ti_debug)[:10]

DatetimeIndex(['2016-01-04 00:00:59.498561536',
               '2016-01-04 23:59:31.161341952',
               '2016-01-06 00:00:20.263075840',
               '2016-01-06 23:58:51.925856256',
               '2016-01-07 23:59:41.027590144',
               '2016-01-09 00:00:30.129324032',
               '2016-01-09 23:59:01.792104448',
               '2016-01-10 23:59:50.893838336',
               '2016-01-12 00:00:39.995572224',
               '2016-01-12 23:59:11.658352640'],
              dtype='datetime64[ns]', freq=None)

In [30]:
pandas.to_datetime(df.date_index)[:10]

0   2016-01-01
1   2016-01-02
2   2016-01-03
3   2016-01-04
4   2016-01-05
5   2016-01-06
6   2016-01-07
7   2016-01-08
8   2016-01-09
9   2016-01-10
Name: date_index, dtype: datetime64[ns]

In [31]:
pandas.to_datetime(torch.Tensor(df.date_index))[:10]

DatetimeIndex(['2016-01-01 00:00:49.632313344',
               '2016-01-01 23:59:21.295093760',
               '2016-01-03 00:00:10.396827648',
               '2016-01-04 00:00:59.498561536',
               '2016-01-04 23:59:31.161341952',
               '2016-01-06 00:00:20.263075840',
               '2016-01-06 23:58:51.925856256',
               '2016-01-07 23:59:41.027590144',
               '2016-01-09 00:00:30.129324032',
               '2016-01-09 23:59:01.792104448'],
              dtype='datetime64[ns]', freq=None)

In [ ]:
import datetime

def adjust_to(ts):
    dte = ts.to_pydatetime()
    dte += datetime.timedelta(hours=1)
    return datetime.datetime(year=dte.year, month=dte.month, day=dte.day, hour=0, minute=0, second=0)


def to_date(ti: numpy.array):
    _ti = pandas.to_datetime(ti)
    ti_ser = pandas.Series(_ti, name="time_index")
    return ti_ser.apply(adjust_to)

ti_ser = to_date(ti_debug)
ti_ser.head()


In [10]:
# df.drop("date_index", axis=1, inplace=True)
# df.head(3)

In [11]:
1451779200000000000
1234567812345678901

1234567812345678901

In [12]:
d = pandas.date_range("2016-01-01", "2018-12-31")
d

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2018-12-22', '2018-12-23', '2018-12-24', '2018-12-25',
               '2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
               '2018-12-30', '2018-12-31'],
              dtype='datetime64[ns]', length=1096, freq='D')

In [13]:
d.astype(numpy.int64)

Int64Index([1451606400000000000, 1451692800000000000, 1451779200000000000,
            1451865600000000000, 1451952000000000000, 1452038400000000000,
            1452124800000000000, 1452211200000000000, 1452297600000000000,
            1452384000000000000,
            ...
            1545436800000000000, 1545523200000000000, 1545609600000000000,
            1545696000000000000, 1545782400000000000, 1545868800000000000,
            1545955200000000000, 1546041600000000000, 1546128000000000000,
            1546214400000000000],
           dtype='int64', length=1096)